In [1]:
from transformers import GPT2LMHeadModel

In [15]:
model_hf = GPT2LMHeadModel.from_pretrained('gpt2')

model_hf_sd = model_hf.state_dict()
for k, v in model_hf_sd.items():
    print(k,"----------------------------------" ,v.shape)

transformer.wte.weight ---------------------------------- torch.Size([50257, 768])
transformer.wpe.weight ---------------------------------- torch.Size([1024, 768])
transformer.h.0.ln_1.weight ---------------------------------- torch.Size([768])
transformer.h.0.ln_1.bias ---------------------------------- torch.Size([768])
transformer.h.0.attn.c_attn.weight ---------------------------------- torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias ---------------------------------- torch.Size([2304])
transformer.h.0.attn.c_proj.weight ---------------------------------- torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias ---------------------------------- torch.Size([768])
transformer.h.0.ln_2.weight ---------------------------------- torch.Size([768])
transformer.h.0.ln_2.bias ---------------------------------- torch.Size([768])
transformer.h.0.mlp.c_fc.weight ---------------------------------- torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias ----------------------------------

In [12]:
from tradeformer.model import TradeFormer, TradeFormerConfig
from tradeformer.tokenizer import MarketTokenizer


with open('./Data/preprocessed_data/train.txt', 'r') as f:
    train_data = f.readlines()


cfg = TradeFormerConfig()
model = TradeFormer(cfg)
tokenizer = MarketTokenizer('./trained_tokenizer/')

In [4]:
import torch

test_inp = torch.randint(0, 100, (5, 8), dtype=torch.long)
model(test_inp).shape

torch.Size([5, 8, 1000])

In [18]:
test_inp_str = [(' '.join(x.split()[:10])) for x in train_data[0:5]]
test_inp_toknized = torch.tensor([tokenizer.encode(test_inp_str[i]) for i in range(len(test_inp_str))])
test_inp_str, test_inp_toknized

(['AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM',
  'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM',
  'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM',
  'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM',
  'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM'],
 tensor([[ 44,  35, 171,  86, 785, 166,  93],
         [ 44,  35, 171,  86, 785, 166,  93],
         [ 44,  35, 171,  86, 785, 166,  93],
         [ 44,  35, 171,  86, 785, 166,  93],
         [ 44,  35, 171,  86, 785, 166,  93]]))

In [19]:
model(test_inp_toknized)

tensor([[[-0.0732, -0.6493, -0.5046,  ..., -0.3713,  0.5752, -0.0057],
         [-0.1772, -0.1031, -0.2421,  ..., -0.9707, -0.2175,  0.1517],
         [-0.5231,  0.6064,  0.1934,  ..., -0.3642, -0.0319,  0.6557],
         ...,
         [ 0.1375, -0.3910, -0.8236,  ..., -0.2809, -0.2844, -0.7719],
         [-0.8966,  0.7704, -0.5022,  ..., -0.4647, -1.1333,  0.4188],
         [ 1.1706, -0.0215,  0.0896,  ..., -0.1421, -0.0074, -0.3911]],

        [[-0.0732, -0.6493, -0.5046,  ..., -0.3713,  0.5752, -0.0057],
         [-0.1772, -0.1031, -0.2421,  ..., -0.9707, -0.2175,  0.1517],
         [-0.5231,  0.6064,  0.1934,  ..., -0.3642, -0.0319,  0.6557],
         ...,
         [ 0.1375, -0.3910, -0.8236,  ..., -0.2809, -0.2844, -0.7719],
         [-0.8966,  0.7704, -0.5022,  ..., -0.4647, -1.1333,  0.4188],
         [ 1.1706, -0.0215,  0.0896,  ..., -0.1421, -0.0074, -0.3911]],

        [[-0.0732, -0.6493, -0.5046,  ..., -0.3713,  0.5752, -0.0057],
         [-0.1772, -0.1031, -0.2421,  ..., -0

In [23]:
import torch.nn.functional as F
#TODO move the generate function into the model class
def generate(tokens, len=5, max_len=20, num_top_samples=10, device='cpu'):
    # TOKENS: B, T
    tokens = tokens.to(device) 
    while tokens.size(1) < max_len:
        with torch.no_grad():
            out = model(tokens)

            logits = out[:, -1, :]
            probs = F.softmax(logits, 1)

            topk_probs, topk_idx = torch.topk(probs, num_top_samples, dim=-1)

            ix = torch.multinomial(topk_probs, 1)
            xcol = torch.gather(topk_idx, -1, ix) #(B, 1)

            tokens = torch.cat((tokens, xcol), dim=1)

    return tokens

out = generate(test_inp_toknized)


In [24]:
test_inp_toknized.shape,  out.shape

(torch.Size([5, 7]), torch.Size([5, 20]))

In [30]:
tokenizer.decode(out.tolist()[0])

'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM AGO AJO BEL AHL AGL AHO AJO AGQ CEM AJO AJL BDL BEL BGM BJO AEM AHL BHN BFL AFL BFO AHL BGL BEM BEO AIO'

In [5]:
with open('Data/preprocessed_data/test.txt', 'r') as f:
    text = f.readlines()

data = ' '.join(text[0].split()[:1000])

In [7]:
from tradeformer import MarketTokenizer

tokenizer = MarketTokenizer('./trained_tokenizer/')


In [9]:
tokens = tokenizer.encode(data)
print(tokens)

[80, 609, 39, 24, 79, 79, 314, 725, 32, 177, 219, 185, 469, 29, 34, 257, 538, 29, 721, 65, 76, 485, 8, 18, 76, 715, 91, 67, 72, 39, 194, 45, 562, 81, 64, 8, 85, 92, 285, 13, 18, 110, 2, 22, 9, 43, 79, 83, 30, 64, 237, 584, 59, 11, 151, 976, 11, 967, 0, 73, 12, 109, 207, 154, 74, 43, 109, 219, 178, 59, 34, 81, 47, 24, 72, 694, 62, 351, 46, 642, 88, 50, 193, 74, 27, 233, 25, 307, 79, 45, 1, 241, 43, 377, 24, 41, 31, 95, 291, 878, 10, 2, 67, 185, 36, 56, 92, 57, 58, 24, 79, 58, 8, 57, 56, 243, 106, 184, 92, 23, 362, 58, 29, 31, 34, 95, 69, 96, 46, 66, 52, 46, 81, 93, 9, 383, 201, 192, 171, 286, 20, 751, 32, 66, 16, 322, 90, 48, 184, 76, 66, 307, 195, 72, 169, 27, 448, 453, 19, 11, 22, 63, 59, 67, 468, 80, 15, 25, 29, 77, 952, 278, 80, 27, 38, 46, 110, 65, 66, 73, 8, 183, 158, 78, 39, 629, 66, 15, 60, 9, 15, 418, 92, 61, 342, 85, 296, 79, 217, 34, 185, 58, 334, 8, 23, 520, 10, 13, 92, 72, 20, 78, 16, 1, 406, 83, 70, 39, 73, 72, 80, 66, 36, 53, 22, 48, 364, 282, 93, 66, 73, 11, 15, 74, 17, 

In [12]:
import torch

buf = torch.tensor(tokens[:24 + 1])
x = buf[:-1].view(4, 6)
y = buf[1:].view(4, 6)
print(x)
print(y)

tensor([[ 80, 609,  39,  24,  79,  79],
        [314, 725,  32, 177, 219, 185],
        [469,  29,  34, 257, 538,  29],
        [721,  65,  76, 485,   8,  18]])
tensor([[609,  39,  24,  79,  79, 314],
        [725,  32, 177, 219, 185, 469],
        [ 29,  34, 257, 538,  29, 721],
        [ 65,  76, 485,   8,  18,  76]])


In [13]:
len(buf)

25

In [1]:
from dataloader import DataLoaderLite

tokenizer_path = './trained_tokenizer/'
dataset_path = './Data/preprocessed_data/token.txt'
B = 4
T = 32
dataloader = DataLoaderLite(tokenizer_path, dataset_path, B, T)

100%|██████████| 12/12 [00:32<00:00,  2.68s/it]

loaded 125796 tokens
1 epoch = 982 batches
